### Code below all in one cell for ease of running whilst operation

In [4]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec  3 12:49:01 2025

@author: Oonagh
"""

## BioEM EOG Project ##

import serial
import time
import keyboard   # pip install keyboard

# ======== CONFIG ========

SERIAL_PORT = 'COM4'       # Windows example: 'COM3' or 'COM4'
BAUDRATE = 9600

SAMPLE_RATE_APPROX = 100.0   # Hz (from Arduino delay)
BLINK_THRESHOLD_DU = 80      # du above baseline (tune this!)
BASELINE_ALPHA = 0.01        # EMA speed for baseline

REFRACTORY_MS = 200          # minimum time between blink detections - will need to be edited
SEQ_WINDOW_MS = 900          # max window for counting blinks in a sequence - will need o be tuned
DOUBLE_DECISION_DELAY_MS = 450  # wait this long to be sure no 3rd blink

# =========================


def send_play_pause():
    print(">> PLAY/PAUSE")
    # Media play/pause key (Windows / many players)
    keyboard.send("play/pause media")


def send_next_track():
    print(">> NEXT TRACK")
    keyboard.send("next track")


def main():
    # Open serial connection to Arduino
    ser = serial.Serial(SERIAL_PORT, BAUDRATE, timeout=1)

    baseline = None
    last_blink_time = 0.0
    in_blink = False  # used to avoid multiple detections per blink

    blink_times = []  # timestamps of recent blinks (seconds)
    is_playing = False

    pending_action = None  # {'type': 'double', 't': timestamp_of_2nd_blink}

    print("Connected to", SERIAL_PORT)
    print("Blink commands:")
    print("  2 blinks: play/pause")
    print("  3 blinks (while playing): skip track\n")

    while True:
        line = ser.readline()
        if not line:
            # no data this cycle
            # still check pending double-blink actions
            now = time.time()
        else:
            try:
                text = line.decode('ascii').strip()
                v = int(text)
            except (UnicodeDecodeError, ValueError):
                continue  # bad line, skip

            now = time.time()

            # Initialize baseline
            if baseline is None:
                baseline = v

            # Exponential moving average baseline
            baseline = (1.0 - BASELINE_ALPHA) * baseline + BASELINE_ALPHA * v
            dev = v - baseline
            mag = abs(dev)

            # Blink detection logic
            if mag > BLINK_THRESHOLD_DU:
                # possible blink
                if not in_blink and (now - last_blink_time) * 1000.0 > REFRACTORY_MS:
                    # register new blink
                    last_blink_time = now
                    in_blink = True

                    # Add to list of recent blinks
                    blink_times.append(now)

                    # Drop old blinks outside sequence window
                    cutoff = now - (SEQ_WINDOW_MS / 1000.0)
                    blink_times = [t for t in blink_times if t >= cutoff]

                    num_recent = len(blink_times)
                    print(f"Blink detected. Recent blinks in window: {num_recent}")

                    # ---- SEQUENCE LOGIC ----
                    if num_recent == 2:
                        # Tentative double-blink, wait to see if a 3rd comes
                        pending_action = {
                            'type': 'double',
                            't': now
                        }

                    elif num_recent >= 3:
                        # Triple (or more) blinks in sequence: treat as triple
                        # Cancel any pending double action:
                        pending_action = None

                        # Only skip if already playing
                        if is_playing:
                            send_next_track()
                        else:
                            print("Triple blink detected, but music not playing -> ignoring skip")
                        # After triple, clear sequence to avoid chaining
                        blink_times = []
            else:
                # signal small again → blink finished
                if in_blink and mag < BLINK_THRESHOLD_DU * 0.5:
                    in_blink = False

        # ---- CHECK PENDING DOUBLE-BLINK ACTION ----
        if pending_action is not None:
            now2 = time.time()
            dt_ms = (now2 - pending_action['t']) * 1000.0

            if dt_ms > DOUBLE_DECISION_DELAY_MS:
                # Time window passed and no triple blink arrived -> commit double
                # Double blink -> toggle play/pause
                if is_playing:
                    send_play_pause()
                    is_playing = False
                    print("State: PAUSED")
                else:
                    send_play_pause()
                    is_playing = True
                    print("State: PLAYING")

                pending_action = None
                # Clear blink history to avoid mixing sequences
                blink_times = []

        # Small sleep to avoid hammering CPU
        time.sleep(0.001)


if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nExiting...")


Connected to COM4
Blink commands:
  2 blinks: play/pause
  3 blinks (while playing): skip track


Exiting...


 ### Run code

In [3]:
#eog_music_control.py

NameError: name 'eog_music_control' is not defined